<a href="https://colab.research.google.com/github/Vanshsingh1203/vanshsingh.github.io/blob/main/Operation_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# OR 6205 – Falcon Die Casting
# Question 1: Trial-and-error Week 1 schedule

import pandas as pd

# -----------------------------
# 1. Input data
# -----------------------------

# Week 1 demand
demand = {
    1: 3500,  # Part 1
    2: 3000,  # Part 2
    3: 4000,  # Part 3
    4: 4000,  # Part 4
    5: 2800   # Part 5
}

# Production rates (units/hour) for feasible (machine, part) pairs
# (machine, part): rate
rates = {
    (1, 1): 40, (1, 4): 60,
    (2, 1): 35, (2, 2): 25,
    (3, 2): 30, (3, 5): 45,
    (4, 2): 35, (4, 3): 50,
    (5, 4): 60, (5, 5): 50
}

# Yield factors by part
yields = {
    1: 0.60,
    2: 0.55,
    3: 0.75,
    4: 0.65,
    5: 0.60
}

# Setup times S_{m,p} in hours
setups = {
    (1, 1): 8, (1, 4): 8,
    (2, 1): 10, (2, 2): 8,
    (3, 2): 10, (3, 5): 24,
    (4, 2): 8, (4, 3): 12,
    (5, 4): 8, (5, 5): 20
}

machines = [1, 2, 3, 4, 5]
parts = [1, 2, 3, 4, 5]
REG_CAP = 120
MAX_TOTAL = 168  # regular + overtime


# -----------------------------
# 2. Effective good-units/hour
# -----------------------------
eff_rate = {}
for (m, p), r in rates.items():
    eff_rate[(m, p)] = r * yields[p]

# -----------------------------
# 3. Trial schedule (hours of production)
#    These are the h_{m,p} we decided by hand.
# -----------------------------
hours = {
    (1, 1): 96.102564,   # h_1,1
    (1, 4): 55.897436,   # h_1,4
    (2, 1): 56.835165,   # h_2,1
    (2, 2): 57.381818,   # h_2,2
    (3, 2): 134.0,       # h_3,2
    (4, 3): 106.666667,  # h_4,3
    (5, 4): 46.666667,   # h_5,4
    (5, 5): 93.333333    # h_5,5
    # all other (m,p) implicitly 0
}

# -----------------------------
# 4. Build detailed part-level table
# -----------------------------
rows = []
for (m, p), hmp in hours.items():
    if hmp <= 1e-6:
        continue
    good_units = eff_rate[(m, p)] * hmp
    rows.append({
        "Machine": m,
        "Part": p,
        "Prod_hours": hmp,
        "Setup_hours": setups[(m, p)],
        "Total_hours_for_part": hmp + setups[(m, p)],
        "Good_units": good_units
    })

part_table = pd.DataFrame(rows)
part_table = part_table.sort_values(["Machine", "Part"]).reset_index(drop=True)

print("Per (machine, part) production plan:\n")
print(part_table.to_string(index=False))

# -----------------------------
# 5. Check total demand satisfaction
# -----------------------------
demand_check = (
    part_table
    .groupby("Part")["Good_units"]
    .sum()
    .rename("Produced")
    .to_frame()
)
demand_check["Demand"] = demand_check.index.map(demand)
demand_check["Gap"] = demand_check["Produced"] - demand_check["Demand"]

print("\nDemand check (Produced vs Demand):\n")
print(demand_check)

# -----------------------------
# 6. Machine-level hours and overtime
# -----------------------------
# sum setup per machine only for parts actually produced
setup_per_machine = (
    part_table
    .groupby("Machine")["Setup_hours"]
    .sum()
    .to_dict()
)

prod_hours_per_machine = (
    part_table
    .groupby("Machine")["Prod_hours"]
    .sum()
    .to_dict()
)

summary_rows = []
for m in machines:
    prod_h = prod_hours_per_machine.get(m, 0.0)
    setup_h = setup_per_machine.get(m, 0.0)
    total_h = prod_h + setup_h
    regular_h = min(REG_CAP, total_h)
    overtime_h = max(0.0, total_h - REG_CAP)
    summary_rows.append({
        "Machine": m,
        "Prod_hours": round(prod_h, 3),
        "Setup_hours": round(setup_h, 3),
        "Total_hours": round(total_h, 3),
        "Regular_hours": round(regular_h, 3),
        "Overtime_hours": round(overtime_h, 3),
        "Within_168_cap": total_h <= MAX_TOTAL
    })

machine_summary = pd.DataFrame(summary_rows).sort_values("Machine")

print("\nMachine-level summary:\n")
print(machine_summary.to_string(index=False))


Per (machine, part) production plan:

 Machine  Part  Prod_hours  Setup_hours  Total_hours_for_part  Good_units
       1     1   96.102564            8            104.102564 2306.461536
       1     4   55.897436            8             63.897436 2180.000004
       2     1   56.835165           10             66.835165 1193.538465
       2     2   57.381818            8             65.381818  788.999998
       3     2  134.000000           10            144.000000 2211.000000
       4     3  106.666667           12            118.666667 4000.000013
       5     4   46.666667            8             54.666667 1820.000013
       5     5   93.333333           20            113.333333 2799.999990

Demand check (Produced vs Demand):

         Produced  Demand           Gap
Part                                   
1     3500.000001    3500  9.999999e-07
2     2999.999998    3000 -2.500000e-06
3     4000.000013    4000  1.250000e-05
4     4000.000017    4000  1.700000e-05
5     2799.999990  

In [2]:
# OR 6205 – Falcon Die Casting
# Question 2: Minimize total overtime using an LP/MIP model

!pip install pulp
import pulp
import pandas as pd

# -----------------------------
# 1. Input data
# -----------------------------
demand = {1:3500, 2:3000, 3:4000, 4:4000, 5:2800}

rates = {
    (1,1):40, (1,4):60,
    (2,1):35, (2,2):25,
    (3,2):30, (3,5):45,
    (4,2):35, (4,3):50,
    (5,4):60, (5,5):50
}

yields = {1:0.60, 2:0.55, 3:0.75, 4:0.65, 5:0.60}

setups = {
    (1,1):8, (1,4):8,
    (2,1):10, (2,2):8,
    (3,2):10, (3,5):24,
    (4,2):8, (4,3):12,
    (5,4):8, (5,5):20
}

machines = [1,2,3,4,5]
parts = [1,2,3,4,5]
REG = 120
MAXOT = 48
BIGM = 10000

# Compute effective rates
eff = {}
for (m,p), r in rates.items():
    eff[(m,p)] = r * yields[p]

# -----------------------------
# 2. Define the MIP model
# -----------------------------
model = pulp.LpProblem("FDC_Q2_Minimize_Overtime", pulp.LpMinimize)

# Variables
h = pulp.LpVariable.dicts("h", (machines, parts), lowBound=0)
x = pulp.LpVariable.dicts("x", (machines, parts), lowBound=0, upBound=1, cat='Binary')
OT = pulp.LpVariable.dicts("OT", machines, lowBound=0, upBound=MAXOT)

# -----------------------------
# 3. Objective: minimize total overtime
# -----------------------------
model += pulp.lpSum([OT[m] for m in machines])

# -----------------------------
# 4. Constraints
# -----------------------------

# Demand satisfaction
for p in parts:
    model += pulp.lpSum(eff[(m,p)] * h[m][p]
                        for m in machines if (m,p) in eff) == demand[p]

# Linking production and setup
for (m,p) in eff:
    model += h[m][p] <= BIGM * x[m][p]

# Machine time constraints
for m in machines:
    model += (
        pulp.lpSum(h[m][p] for p in parts if (m,p) in eff) +
        pulp.lpSum(setups[(m,p)] * x[m][p] for p in parts if (m,p) in setups)
        == REG + OT[m]
    )

# -----------------------------
# 5. Solve
# -----------------------------
model.solve(pulp.PULP_CBC_CMD(msg=1))

print("Status:", pulp.LpStatus[model.status])

# -----------------------------
# 6. Output solution tables
# -----------------------------
rows = []
for m in machines:
    for p in parts:
        if (m,p) in eff:
            if h[m][p].value() > 1e-6:
                rows.append({
                    "Machine": m,
                    "Part": p,
                    "Hours": h[m][p].value(),
                    "Setup": setups[(m,p)],
                })

df = pd.DataFrame(rows)
print("\nProduction Plan (nonzero h_mp):")
print(df.to_string(index=False))

# Machine summary
machine_summary = []
for m in machines:
    prod = sum(h[m][p].value() for p in parts if (m,p) in eff)
    setup = sum(setups[(m,p)] * x[m][p].value() for p in parts if (m,p) in setups)
    total = prod + setup
    machine_summary.append({
        "Machine": m,
        "Prod_hours": prod,
        "Setup_hours": setup,
        "Total_hours": total,
        "OT": OT[m].value()
    })

machine_summary = pd.DataFrame(machine_summary)
print("\nMachine Summary:")
print(machine_summary.to_string(index=False))


Status: Optimal

Production Plan (nonzero h_mp):
 Machine  Part      Hours  Setup
       1     1  49.583333      8
       1     4  55.897436      8
       2     1 110.000000     10
       3     2 133.595960     10
       4     2  41.333333      8
       4     3 106.666670     12
       5     4  46.666667      8
       5     5  93.333333     20

Machine Summary:
 Machine  Prod_hours  Setup_hours  Total_hours        OT
       1  105.480769         16.0   121.480769  1.480769
       2  110.000000         10.0   120.000000  0.000000
       3  133.595960         10.0   143.595960 23.595960
       4  148.000003         20.0   168.000003 48.000000
       5  140.000000         28.0   168.000000 48.000000


In [3]:
# OR 6205 – Falcon Die Casting
# Question 3: Minimize maximum overtime (duration of overtime production)

!pip install pulp
import pulp
import pandas as pd

# -----------------------------
# 1. Input data (same as Q2)
# -----------------------------
demand = {1:3500, 2:3000, 3:4000, 4:4000, 5:2800}

rates = {
    (1,1):40, (1,4):60,
    (2,1):35, (2,2):25,
    (3,2):30, (3,5):45,
    (4,2):35, (4,3):50,
    (5,4):60, (5,5):50
}

yields = {1:0.60, 2:0.55, 3:0.75, 4:0.65, 5:0.60}

setups = {
    (1,1):8, (1,4):8,
    (2,1):10, (2,2):8,
    (3,2):10, (3,5):24,
    (4,2):8, (4,3):12,
    (5,4):8, (5,5):20
}

machines = [1,2,3,4,5]
parts = [1,2,3,4,5]
REG = 120
MAXOT = 48
BIGM = 10000

# Effective good-unit rates
eff = {(m,p): rates[(m,p)] * yields[p] for (m,p) in rates.keys()}

# -----------------------------
# 2. Define MIP model
# -----------------------------
model = pulp.LpProblem("FDC_Q3_Minimize_Max_Overtime", pulp.LpMinimize)

# Decision variables
h = pulp.LpVariable.dicts("h", (machines, parts), lowBound=0)
x = pulp.LpVariable.dicts("x", (machines, parts), lowBound=0, upBound=1, cat='Binary')
OT = pulp.LpVariable.dicts("OT", machines, lowBound=0, upBound=MAXOT)

# New variable: maximum overtime
W = pulp.LpVariable("W", lowBound=0)

# -----------------------------
# 3. Objective: minimize W
# -----------------------------
model += W

# -----------------------------
# 4. Constraints
# -----------------------------

# Demand satisfaction
for p in parts:
    model += pulp.lpSum(eff[(m,p)] * h[m][p]
                        for m in machines if (m,p) in eff) == demand[p], f"demand_p{p}"

# Linking production and setup
for (m,p) in eff:
    model += h[m][p] <= BIGM * x[m][p], f"prod_setup_link_m{m}_p{p}"

# Machine time balance
for m in machines:
    model += (
        pulp.lpSum(h[m][p] for p in parts if (m,p) in eff) +
        pulp.lpSum(setups[(m,p)] * x[m][p] for p in parts if (m,p) in setups)
        == REG + OT[m]
    ), f"time_balance_m{m}"

# Overtime bound already enforced by variable definition: 0 <= OT[m] <= MAXOT

# Max-overtime constraints: OT_m <= W
for m in machines:
    model += OT[m] <= W, f"max_ot_m{m}"

# -----------------------------
# 5. Solve
# -----------------------------
model.solve(pulp.PULP_CBC_CMD(msg=1))
print("Status:", pulp.LpStatus[model.status])
print("Optimal maximum overtime W* =", W.value())

# -----------------------------
# 6. Extract solution
# -----------------------------
rows = []
for m in machines:
    for p in parts:
        if (m,p) in eff and h[m][p].value() > 1e-6:
            rows.append({
                "Machine": m,
                "Part": p,
                "Hours": h[m][p].value(),
                "Setup": setups[(m,p)]
            })

df_plan = pd.DataFrame(rows).sort_values(["Machine","Part"])
print("\nProduction Plan (nonzero h_mp):")
print(df_plan.to_string(index=False))

# Machine summary
machine_rows = []
for m in machines:
    prod = sum(h[m][p].value() for p in parts if (m,p) in eff)
    setup = sum(setups[(m,p)] * x[m][p].value() for p in parts if (m,p) in setups)
    total = prod + setup
    machine_rows.append({
        "Machine": m,
        "Prod_hours": prod,
        "Setup_hours": setup,
        "Total_hours": total,
        "OT": OT[m].value()
    })

df_mach = pd.DataFrame(machine_rows).sort_values("Machine")
print("\nMachine Summary:")
print(df_mach.to_string(index=False))


Status: Optimal
Optimal maximum overtime W* = 27.195442

Production Plan (nonzero h_mp):
 Machine  Part      Hours  Setup
       1     1  54.493448      8
       1     4  76.701994      8
       2     1 104.388440     10
       2     2  24.807002      8
       3     2 137.195440     10
       4     2  20.528775      8
       4     3 106.666670     12
       5     4  25.862109      8
       5     5  93.333333     20

Machine Summary:
 Machine  Prod_hours  Setup_hours  Total_hours        OT
       1  131.195442         16.0   147.195442 27.195442
       2  129.195442         18.0   147.195442 27.195442
       3  137.195440         10.0   147.195440 27.195442
       4  127.195445         20.0   147.195445 27.195442
       5  119.195442         28.0   147.195442 27.195442


In [4]:
# OR 6205 – Falcon Die Casting
# Question 4: Minimize total overtime cost (machines + support staff)

!pip install pulp
import pulp
import pandas as pd

# -----------------------------
# 1. Input data (same as Q2/Q3)
# -----------------------------
demand = {1:3500, 2:3000, 3:4000, 4:4000, 5:2800}

rates = {
    (1,1):40, (1,4):60,
    (2,1):35, (2,2):25,
    (3,2):30, (3,5):45,
    (4,2):35, (4,3):50,
    (5,4):60, (5,5):50
}

yields = {1:0.60, 2:0.55, 3:0.75, 4:0.65, 5:0.60}

setups = {
    (1,1):8, (1,4):8,
    (2,1):10, (2,2):8,
    (3,2):10, (3,5):24,
    (4,2):8, (4,3):12,
    (5,4):8, (5,5):20
}

machines = [1,2,3,4,5]
parts = [1,2,3,4,5]
REG = 120
MAXOT = 48
BIGM = 10000

# Costs
c_machine = 30.0  # $ per machine overtime hour
c_support = 40.0  # $ per hour of max overtime (support staff)

# Effective good-unit rates
eff = {(m,p): rates[(m,p)] * yields[p] for (m,p) in rates.keys()}

# -----------------------------
# 2. Define MIP model
# -----------------------------
model = pulp.LpProblem("FDC_Q4_Minimize_OT_Cost", pulp.LpMinimize)

# Decision variables
h = pulp.LpVariable.dicts("h", (machines, parts), lowBound=0)
x = pulp.LpVariable.dicts("x", (machines, parts), lowBound=0, upBound=1, cat='Binary')
OT = pulp.LpVariable.dicts("OT", machines, lowBound=0, upBound=MAXOT)
W = pulp.LpVariable("W", lowBound=0)

# -----------------------------
# 3. Objective: minimize total OT cost
# Z = 30 * sum_m OT_m + 40 * W
# -----------------------------
model += (
    c_machine * pulp.lpSum(OT[m] for m in machines)
    + c_support * W
), "Total_Overtime_Cost"

# -----------------------------
# 4. Constraints
# -----------------------------

# Demand satisfaction
for p in parts:
    model += pulp.lpSum(eff[(m,p)] * h[m][p]
                        for m in machines if (m,p) in eff) == demand[p], f"demand_p{p}"

# Link production to setup
for (m,p) in eff:
    model += h[m][p] <= BIGM * x[m][p], f"prod_setup_link_m{m}_p{p}"

# Machine time balance
for m in machines:
    model += (
        pulp.lpSum(h[m][p] for p in parts if (m,p) in eff) +
        pulp.lpSum(setups[(m,p)] * x[m][p] for p in parts if (m,p) in setups)
        == REG + OT[m]
    ), f"time_balance_m{m}"

# Max-overtime constraints
for m in machines:
    model += OT[m] <= W, f"max_ot_m{m}"

# (OT bounds are already enforced by variable definitions)

# -----------------------------
# 5. Solve
# -----------------------------
model.solve(pulp.PULP_CBC_CMD(msg=1))
print("Status:", pulp.LpStatus[model.status])

W_star = W.value()
total_OT = sum(OT[m].value() for m in machines)
total_cost = c_machine * total_OT + c_support * W_star

print(f"Optimal W* (max overtime) = {W_star:.4f} hours")
print(f"Total overtime across machines = {total_OT:.4f} hours")
print(f"Total overtime cost = ${total_cost:.2f}")

# -----------------------------
# 6. Extract solution
# -----------------------------
rows = []
for m in machines:
    for p in parts:
        if (m,p) in eff and h[m][p].value() > 1e-6:
            rows.append({
                "Machine": m,
                "Part": p,
                "Hours": h[m][p].value(),
                "Setup_hours": setups[(m,p)]
            })

df_plan = pd.DataFrame(rows).sort_values(["Machine","Part"])
print("\nProduction Plan (nonzero h_mp):")
print(df_plan.to_string(index=False))

# Machine summary
machine_rows = []
for m in machines:
    prod = sum(h[m][p].value() for p in parts if (m,p) in eff)
    setup = sum(setups[(m,p)] * x[m][p].value() for p in parts if (m,p) in setups)
    total = prod + setup
    machine_rows.append({
        "Machine": m,
        "Prod_hours": prod,
        "Setup_hours": setup,
        "Total_hours": total,
        "OT": OT[m].value()
    })

df_mach = pd.DataFrame(machine_rows).sort_values("Machine")
print("\nMachine Summary:")
print(df_mach.to_string(index=False))


Status: Optimal
Optimal W* (max overtime) = 36.7366 hours
Total overtime across machines = 122.9540 hours
Total overtime cost = $5158.08

Production Plan (nonzero h_mp):
 Machine  Part      Hours  Setup_hours
       1     1  49.583333            8
       1     4  91.153263            8
       2     1 110.000000           10
       3     2 146.736600           10
       4     2  30.069930            8
       4     3 106.666670           12
       5     4  11.410839            8
       5     5  93.333333           20

Machine Summary:
 Machine  Prod_hours  Setup_hours  Total_hours        OT
       1  140.736596         16.0   156.736596 36.736597
       2  110.000000         10.0   120.000000  0.000000
       3  146.736600         10.0   156.736600 36.736597
       4  136.736600         20.0   156.736600 36.736597
       5  104.744172         28.0   132.744172 12.744172


In [6]:
# OR 6205 – Falcon Die Casting
# Question 5: Minimize total overtime with initial setups already done

!pip install pulp
import pulp
import pandas as pd

# -----------------------------
# 1. Input data (same as before)
# -----------------------------
demand = {1:3500, 2:3000, 3:4000, 4:4000, 5:2800}

rates = {
    (1,1):40, (1,4):60,
    (2,1):35, (2,2):25,
    (3,2):30, (3,5):45,
    (4,2):35, (4,3):50,
    (5,4):60, (5,5):50
}

yields = {1:0.60, 2:0.55, 3:0.75, 4:0.65, 5:0.60}

setups = {
    (1,1):8, (1,4):8,
    (2,1):10, (2,2):8,
    (3,2):10, (3,5):24,
    (4,2):8, (4,3):12,
    (5,4):8, (5,5):20
}

machines = [1,2,3,4,5]
parts = [1,2,3,4,5]
REG = 120
MAXOT = 48
BIGM = 10000

# Initial part on each machine (given in the question)
initial_part = {
    1: 1,   # M1 -> P1
    2: 2,   # M2 -> P2
    3: 5,   # M3 -> P5
    4: 3,   # M4 -> P3
    5: 4    # M5 -> P4
}

# Effective good-unit rates
eff = {(m,p): rates[(m,p)] * yields[p] for (m,p) in rates.keys()}

# -----------------------------
# 2. Define MIP model
# -----------------------------
model = pulp.LpProblem("FDC_Q5_Minimize_Overtime_InitialSetup", pulp.LpMinimize)

# Decision variables
h = pulp.LpVariable.dicts("h", (machines, parts), lowBound=0)
x = pulp.LpVariable.dicts("x", (machines, parts), lowBound=0, upBound=1, cat='Binary')
OT = pulp.LpVariable.dicts("OT", machines, lowBound=0, upBound=MAXOT)

# -----------------------------
# 3. Objective: minimize total overtime
# -----------------------------
model += pulp.lpSum(OT[m] for m in machines), "Total_Overtime"

# -----------------------------
# 4. Constraints
# -----------------------------

# Demand satisfaction
for p in parts:
    model += pulp.lpSum(
        eff[(m,p)] * h[m][p]
        for m in machines if (m,p) in eff
    ) == demand[p], f"demand_p{p}"

# Link production and setup
for (m,p) in eff:
    model += h[m][p] <= BIGM * x[m][p], f"prod_setup_link_m{m}_p{p}"

# Machine time balance with initial setup free
for m in machines:
    # Standard setup term
    setup_term = pulp.lpSum(setups[(m,p)] * x[m][p]
                             for p in parts if (m,p) in setups)
    # Subtract initial setup (if that machine-part exists)
    init_p = initial_part[m]
    if (m, init_p) in setups:
        setup_term = setup_term - setups[(m, init_p)] * x[m][init_p]

    model += (
        pulp.lpSum(h[m][p] for p in parts if (m,p) in eff) +
        setup_term
        == REG + OT[m]
    ), f"time_balance_m{m}"

# -----------------------------
# 5. Solve
# -----------------------------
model.solve(pulp.PULP_CBC_CMD(msg=1))
print("Status:", pulp.LpStatus[model.status])

total_OT = sum(OT[m].value() for m in machines)
print(f"Total overtime across machines = {total_OT:.4f} hours")

# -----------------------------
# 6. Extract solution
# -----------------------------
rows = []
for m in machines:
    for p in parts:
        if (m,p) in eff and h[m][p].value() > 1e-6:
            rows.append({
                "Machine": m,
                "Part": p,
                "Hours": h[m][p].value(),
                # Show the actual setup hours charged in time balance
                "Setup_hours_nominal": setups.get((m,p), 0.0),
                "Is_initial_part": int(p == initial_part[m])
            })

df_plan = pd.DataFrame(rows).sort_values(["Machine","Part"])
print("\nProduction Plan (nonzero h_mp):")
print(df_plan.to_string(index=False))

# Machine summary
machine_rows = []
for m in machines:
    prod = sum(h[m][p].value() for p in parts if (m,p) in eff)

    # Compute charged setup (nominal minus initial)
    charged_setup = 0.0
    for p in parts:
        if (m,p) in setups:
            val = setups[(m,p)] * x[m][p].value()
            if p == initial_part[m]:
                val -= setups[(m,p)] * x[m][p].value()
            charged_setup += val

    total = prod + charged_setup
    machine_rows.append({
        "Machine": m,
        "Prod_hours": prod,
        "Setup_hours_charged": charged_setup,
        "Total_hours": total,
        "OT": OT[m].value()
    })

df_mach = pd.DataFrame(machine_rows).sort_values("Machine")
print("\nMachine Summary:")
print(df_mach.to_string(index=False))


Status: Optimal
Total overtime across machines = 88.3671 hours

Production Plan (nonzero h_mp):
 Machine  Part      Hours  Setup_hours_nominal  Is_initial_part
       1     1 145.833330                    8                1
       2     2 120.000000                    8                1
       3     2  19.595960                   10                0
       3     5  90.404040                   24                1
       4     2  53.333333                    8                0
       4     3 106.666670                   12                1
       5     4 102.564100                    8                1
       5     5  11.969697                   20                0

Machine Summary:
 Machine  Prod_hours  Setup_hours_charged  Total_hours        OT
       1  145.833330                  0.0   145.833330 25.833333
       2  120.000000                  0.0   120.000000  0.000000
       3  110.000000                 10.0   120.000000  0.000000
       4  160.000003                  8.0   168.00

In [7]:
# OR 6205 – Falcon Die Casting
# Question 6: Week 11 schedule with inventory and backlog costs

!pip install pulp
import pulp
import pandas as pd

# -----------------------------
# 1. Input data
# -----------------------------

# Week 11 demand
demand = {1:4500, 2:4000, 3:5000, 4:5000, 5:3800}

rates = {
    (1,1):40, (1,4):60,
    (2,1):35, (2,2):25,
    (3,2):30, (3,5):45,
    (4,2):35, (4,3):50,
    (5,4):60, (5,5):50
}

yields = {1:0.60, 2:0.55, 3:0.75, 4:0.65, 5:0.60}

setups = {
    (1,1):8, (1,4):8,
    (2,1):10, (2,2):8,
    (3,2):10, (3,5):24,
    (4,2):8, (4,3):12,
    (5,4):8, (5,5):20
}

machines = [1,2,3,4,5]
parts = [1,2,3,4,5]
REG = 120
MAXOT = 48
BIGM = 10000

# Costs
c_ot = 30.0     # $ per overtime hour (machine)
c_inv = 2.0     # $ per unit of ending inventory
c_bk  = 3.0     # $ per unit of backlog

# Effective good-unit rates
eff = {(m,p): rates[(m,p)] * yields[p] for (m,p) in rates.keys()}

# -----------------------------
# 2. Define MIP model
# -----------------------------
model = pulp.LpProblem("FDC_Q6_Week11_Backlog_Inventory", pulp.LpMinimize)

# Decision variables
h = pulp.LpVariable.dicts("h", (machines, parts), lowBound=0)
x = pulp.LpVariable.dicts("x", (machines, parts), lowBound=0, upBound=1, cat='Binary')
OT = pulp.LpVariable.dicts("OT", machines, lowBound=0, upBound=MAXOT)

I = pulp.LpVariable.dicts("Inv", parts, lowBound=0)   # ending inventory
B = pulp.LpVariable.dicts("Backlog", parts, lowBound=0)  # unmet demand

# -----------------------------
# 3. Objective: total cost
# Z = 30 * sum_m OT[m] + 2 * sum_p I[p] + 3 * sum_p B[p]
# -----------------------------
model += (
    c_ot * pulp.lpSum(OT[m] for m in machines)
    + c_inv * pulp.lpSum(I[p] for p in parts)
    + c_bk  * pulp.lpSum(B[p] for p in parts)
), "Total_Cost"

# -----------------------------
# 4. Constraints
# -----------------------------

# Demand / flow balance for each part
for p in parts:
    model += (
        pulp.lpSum(eff[(m,p)] * h[m][p]
                   for m in machines if (m,p) in eff)
        - I[p] + B[p]
        == demand[p]
    ), f"flow_balance_p{p}"

# Link production to setup
for (m,p) in eff:
    model += h[m][p] <= BIGM * x[m][p], f"prod_setup_link_m{m}_p{p}"

# Machine time balance
for m in machines:
    model += (
        pulp.lpSum(h[m][p] for p in parts if (m,p) in eff) +
        pulp.lpSum(setups[(m,p)] * x[m][p] for p in parts if (m,p) in setups)
        == REG + OT[m]
    ), f"time_balance_m{m}"

# OT bounds already in variable definitions (0 <= OT[m] <= MAXOT)

# -----------------------------
# 5. Solve
# -----------------------------
model.solve(pulp.PULP_CBC_CMD(msg=1))
print("Status:", pulp.LpStatus[model.status])

OT_total = sum(OT[m].value() for m in machines)
I_total = sum(I[p].value() for p in parts)
B_total = sum(B[p].value() for p in parts)

total_cost = (
    c_ot * OT_total +
    c_inv * I_total +
    c_bk  * B_total
)

print(f"\nTotal overtime hours = {OT_total:.4f}")
print(f"Total ending inventory (units) = {I_total:.4f}")
print(f"Total backlog (units) = {B_total:.4f}")
print(f"Total cost = ${total_cost:.2f}")

# -----------------------------
# 6. Detailed outputs
# -----------------------------
# Part-level summary
part_rows = []
for p in parts:
    prod_p = sum(eff[(m,p)] * h[m][p].value()
                 for m in machines if (m,p) in eff)
    part_rows.append({
        "Part": p,
        "Demand": demand[p],
        "Produced_good_units": prod_p,
        "Inventory": I[p].value(),
        "Backlog": B[p].value()
    })

df_parts = pd.DataFrame(part_rows).sort_values("Part")
print("\nPart-level summary:")
print(df_parts.to_string(index=False))

# Machine summary
mach_rows = []
for m in machines:
    prod_h = sum(h[m][p].value() for p in parts if (m,p) in eff)
    setup_h = sum(setups[(m,p)] * x[m][p].value() for p in parts if (m,p) in setups)
    total_h = prod_h + setup_h
    mach_rows.append({
        "Machine": m,
        "Prod_hours": prod_h,
        "Setup_hours": setup_h,
        "Total_hours": total_h,
        "OT": OT[m].value()
    })

df_mach = pd.DataFrame(mach_rows).sort_values("Machine")
print("\nMachine summary:")
print(df_mach.to_string(index=False))


Status: Optimal

Total overtime hours = 240.0000
Total ending inventory (units) = 0.0000
Total backlog (units) = 1401.5898
Total cost = $11404.77

Part-level summary:
 Part  Demand  Produced_good_units  Inventory    Backlog
    1    4500          4209.076920        0.0  290.92308
    2    4000          2889.333340        0.0 1110.66670
    3    5000          4999.999875        0.0    0.00000
    4    5000          4999.999797        0.0    0.00000
    5    3800          3800.000100        0.0    0.00000

Machine summary:
 Machine  Prod_hours  Setup_hours  Total_hours   OT
       1  151.999995         16.0   167.999995 48.0
       2  158.000000         10.0   168.000000 48.0
       3  158.000000         10.0   168.000000 48.0
       4  147.999997         20.0   167.999997 48.0
       5  140.000003         28.0   168.000003 48.0


In [8]:
# OR 6205 – Falcon Die Casting
# Question 7: Two-week scheduling (Weeks 1 & 2) with setup carry-over

!pip install pulp
import pulp
import pandas as pd

# -----------------------------
# 1. Data
# -----------------------------
machines = [1,2,3,4,5]
parts = [1,2,3,4,5]
weeks = [1,2]

# Weekly demand (using Week 1 and Week 2 from Table 1)
demand = {
    (1,1): 3500, (2,1): 3000, (3,1): 4000, (4,1): 4000, (5,1): 2800,
    (1,2): 3000, (2,2): 2800, (3,2): 4000, (4,2): 4300, (5,2): 2800
}

# Production rates (units/hour)
rates = {
    (1,1):40, (1,4):60,
    (2,1):35, (2,2):25,
    (3,2):30, (3,5):45,
    (4,2):35, (4,3):50,
    (5,4):60, (5,5):50
}

# Yields
yields = {1:0.60, 2:0.55, 3:0.75, 4:0.65, 5:0.60}

# Setup times
setups = {
    (1,1):8, (1,4):8,
    (2,1):10, (2,2):8,
    (3,2):10, (3,5):24,
    (4,2):8, (4,3):12,
    (5,4):8, (5,5):20
}

REG = 120
MAXOT = 48
BIGM = 10000

# Effective good-unit rates
eff = {(m,p): rates[(m,p)] * yields[p] for (m,p) in rates.keys()}

# -----------------------------
# 2. Model
# -----------------------------
model = pulp.LpProblem("FDC_Q7_TwoWeek_SetupCarryover", pulp.LpMinimize)

# Variables
h = pulp.LpVariable.dicts("h", (machines, parts, weeks), lowBound=0)
x = pulp.LpVariable.dicts("x", (machines, parts, weeks), lowBound=0, upBound=1, cat='Binary')
OT = pulp.LpVariable.dicts("OT", (machines, weeks), lowBound=0, upBound=MAXOT)
c = pulp.LpVariable.dicts("carry", (machines, parts), lowBound=0, upBound=1, cat='Binary')

# -----------------------------
# 3. Objective: minimize total overtime over both weeks
# -----------------------------
model += pulp.lpSum(OT[m][t] for m in machines for t in weeks)

# -----------------------------
# 4. Constraints
# -----------------------------

# Demand satisfaction each week
for t in weeks:
    for p in parts:
        model += (
            pulp.lpSum(eff[(m,p)] * h[m][p][t]
                       for m in machines if (m,p) in eff)
            == demand[(p,t)]
        ), f"demand_p{p}_w{t}"

# Production only if setup used that week
for (m,p) in eff:
    for t in weeks:
        model += h[m][p][t] <= BIGM * x[m][p][t], f"prod_link_m{m}_p{p}_w{t}"

# Machine time balance, Week 1
for m in machines:
    model += (
        pulp.lpSum(h[m][p][1] for p in parts if (m,p) in eff) +
        pulp.lpSum(setups[(m,p)] * x[m][p][1] for p in parts if (m,p) in setups)
        == REG + OT[m][1]
    ), f"time_balance_m{m}_w1"

# Machine time balance, Week 2 with setup carry-over
for m in machines:
    model += (
        pulp.lpSum(h[m][p][2] for p in parts if (m,p) in eff) +
        pulp.lpSum(setups[(m,p)] * (x[m][p][2] - c[m][p])
                   for p in parts if (m,p) in setups)
        == REG + OT[m][2]
    ), f"time_balance_m{m}_w2"

# Carry-over logic: c_mp = 1 only if part used in both weeks
for (m,p) in eff:
    model += c[m][p] <= x[m][p][1], f"carry_le_week1_m{m}_p{p}"
    model += c[m][p] <= x[m][p][2], f"carry_le_week2_m{m}_p{p}"
    model += c[m][p] >= x[m][p][1] + x[m][p][2] - 1, f"carry_ge_both_m{m}_p{p}"

# -----------------------------
# 5. Solve
# -----------------------------
model.solve(pulp.PULP_CBC_CMD(msg=1))
print("Status:", pulp.LpStatus[model.status])

total_OT = sum(OT[m][t].value() for m in machines for t in weeks)
print(f"Total overtime over both weeks = {total_OT:.4f} hours")

# -----------------------------
# 6. Output: part-week summary
# -----------------------------
rows_parts = []
for t in weeks:
    for p in parts:
        produced = sum(
            eff[(m,p)] * h[m][p][t].value()
            for m in machines if (m,p) in eff
        )
        rows_parts.append({
            "Week": t,
            "Part": p,
            "Demand": demand[(p,t)],
            "Produced_good_units": produced
        })

df_parts = pd.DataFrame(rows_parts).sort_values(["Week","Part"])
print("\nPart-week production summary:")
print(df_parts.to_string(index=False))

# Machine-week summary
rows_mach = []
for m in machines:
    for t in weeks:
        prod_h = sum(h[m][p][t].value() for p in parts if (m,p) in eff)
        setup_h = sum(setups[(m,p)] * x[m][p][t].value()
                      for p in parts if (m,p) in setups)
        if t == 2:
            # subtract carried setup to see effective charged setup
            setup_h -= sum(setups[(m,p)] * c[m][p].value()
                           for p in parts if (m,p) in setups)
        total_h = prod_h + setup_h
        rows_mach.append({
            "Machine": m,
            "Week": t,
            "Prod_hours": prod_h,
            "Setup_hours_effective": setup_h,
            "Total_hours": total_h,
            "OT": OT[m][t].value()
        })

df_mach = pd.DataFrame(rows_mach).sort_values(["Machine","Week"])
print("\nMachine-week summary:")
print(df_mach.to_string(index=False))


Status: Optimal
Total overtime over both weeks = 142.6590 hours

Part-week production summary:
 Week  Part  Demand  Produced_good_units
    1     1    3500          3499.999992
    1     2    3000          3000.000000
    1     3    4000          4000.000125
    1     4    4000          4000.000017
    1     5    2800          2799.999990
    2     1    3000          2999.999964
    2     2    2800          2800.000010
    2     3    4000          4000.000125
    2     4    4300          4300.000029
    2     5    2800          2799.999990

Machine-week summary:
 Machine  Week  Prod_hours  Setup_hours_effective  Total_hours        OT
       1     1  105.480769                   16.0   121.480769  1.480769
       1     2  120.000000                    0.0   120.000000  0.000000
       2     1  110.000000                   10.0   120.000000  0.000000
       2     2  111.999999                    8.0   119.999999  0.000000
       3     1  133.595960                   10.0   143.595960 23.

In [9]:
# OR 6205 – Falcon Die Casting
# Question 8: Two-week scheduling with setup carry-over + inventory between weeks

!pip install pulp
import pulp
import pandas as pd

# -----------------------------
# 1. Data
# -----------------------------
machines = [1,2,3,4,5]
parts = [1,2,3,4,5]
weeks = [1,2]

# Weekly demand (Week 1 & Week 2 from Table 1)
demand = {
    (1,1): 3500, (2,1): 3000, (3,1): 4000, (4,1): 4000, (5,1): 2800,
    (1,2): 3000, (2,2): 2800, (3,2): 4000, (4,2): 4300, (5,2): 2800
}

# Production rates (units/hour)
rates = {
    (1,1):40, (1,4):60,
    (2,1):35, (2,2):25,
    (3,2):30, (3,5):45,
    (4,2):35, (4,3):50,
    (5,4):60, (5,5):50
}

# Yields
yields = {1:0.60, 2:0.55, 3:0.75, 4:0.65, 5:0.60}

# Setup times
setups = {
    (1,1):8, (1,4):8,
    (2,1):10, (2,2):8,
    (3,2):10, (3,5):24,
    (4,2):8, (4,3):12,
    (5,4):8, (5,5):20
}

REG = 120
MAXOT = 48
BIGM = 10000

# Effective good-unit rates
eff = {(m,p): rates[(m,p)] * yields[p] for (m,p) in rates.keys()}

# -----------------------------
# 2. Model
# -----------------------------
model = pulp.LpProblem("FDC_Q8_TwoWeek_Inventory", pulp.LpMinimize)

# Decision variables
h = pulp.LpVariable.dicts("h", (machines, parts, weeks), lowBound=0)
x = pulp.LpVariable.dicts("x", (machines, parts, weeks), lowBound=0, upBound=1, cat='Binary')
OT = pulp.LpVariable.dicts("OT", (machines, weeks), lowBound=0, upBound=MAXOT)
c = pulp.LpVariable.dicts("carry", (machines, parts), lowBound=0, upBound=1, cat='Binary')
Inv = pulp.LpVariable.dicts("Inv", parts, lowBound=0)  # inventory carried from week 1 to week 2

# -----------------------------
# 3. Objective: minimize total overtime over both weeks
# -----------------------------
model += pulp.lpSum(OT[m][t] for m in machines for t in weeks), "Total_Overtime"

# -----------------------------
# 4. Constraints
# -----------------------------

# Demand / flow balance with inventory between weeks
for p in parts:
    # Week 1: production = demand1 + inventory
    model += (
        pulp.lpSum(eff[(m,p)] * h[m][p][1]
                   for m in machines if (m,p) in eff)
        == demand[(p,1)] + Inv[p]
    ), f"flow_p{p}_w1"

    # Week 2: production + inventory = demand2
    model += (
        pulp.lpSum(eff[(m,p)] * h[m][p][2]
                   for m in machines if (m,p) in eff)
        + Inv[p]
        == demand[(p,2)]
    ), f"flow_p{p}_w2"

# Production only if setup used in that week
for (m,p) in eff:
    for t in weeks:
        model += h[m][p][t] <= BIGM * x[m][p][t], f"prod_link_m{m}_p{p}_w{t}"

# Machine time balance, Week 1 (full setup charged)
for m in machines:
    model += (
        pulp.lpSum(h[m][p][1] for p in parts if (m,p) in eff) +
        pulp.lpSum(setups[(m,p)] * x[m][p][1] for p in parts if (m,p) in setups)
        == REG + OT[m][1]
    ), f"time_balance_m{m}_w1"

# Machine time balance, Week 2 (setup minus carry-over)
for m in machines:
    model += (
        pulp.lpSum(h[m][p][2] for p in parts if (m,p) in eff) +
        pulp.lpSum(setups[(m,p)] * (x[m][p][2] - c[m][p])
                   for p in parts if (m,p) in setups)
        == REG + OT[m][2]
    ), f"time_balance_m{m}_w2"

# Carry-over logic: c_mp = 1 only if part used in both weeks
for (m,p) in eff:
    model += c[m][p] <= x[m][p][1], f"carry_le_w1_m{m}_p{p}"
    model += c[m][p] <= x[m][p][2], f"carry_le_w2_m{m}_p{p}"
    model += c[m][p] >= x[m][p][1] + x[m][p][2] - 1, f"carry_ge_both_m{m}_p{p}"

# -----------------------------
# 5. Solve
# -----------------------------
model.solve(pulp.PULP_CBC_CMD(msg=1))
print("Status:", pulp.LpStatus[model.status])

total_OT = sum(OT[m][t].value() for m in machines for t in weeks)
print(f"\nTotal overtime over both weeks = {total_OT:.4f} hours")

# -----------------------------
# 6. Part-level summary (production and inventory)
# -----------------------------
rows_parts = []
for p in parts:
    prod_w1 = sum(eff[(m,p)] * h[m][p][1].value()
                  for m in machines if (m,p) in eff)
    prod_w2 = sum(eff[(m,p)] * h[m][p][2].value()
                  for m in machines if (m,p) in eff)
    rows_parts.append({
        "Part": p,
        "Demand_week1": demand[(p,1)],
        "Demand_week2": demand[(p,2)],
        "Prod_w1_good": prod_w1,
        "Prod_w2_good": prod_w2,
        "Inventory_carry": Inv[p].value()
    })

df_parts = pd.DataFrame(rows_parts).sort_values("Part")
print("\nPart-level summary (with inventory):")
print(df_parts.to_string(index=False))

# -----------------------------
# 7. Machine-week summary
# -----------------------------
rows_mach = []
for m in machines:
    for t in weeks:
        prod_h = sum(h[m][p][t].value() for p in parts if (m,p) in eff)
        setup_h = sum(setups[(m,p)] * x[m][p][t].value()
                      for p in parts if (m,p) in setups)
        if t == 2:
            setup_h -= sum(setups[(m,p)] * c[m][p].value()
                           for p in parts if (m,p) in setups)
        total_h = prod_h + setup_h
        rows_mach.append({
            "Machine": m,
            "Week": t,
            "Prod_hours": prod_h,
            "Setup_hours_effective": setup_h,
            "Total_hours": total_h,
            "OT": OT[m][t].value()
        })

df_mach = pd.DataFrame(rows_mach).sort_values(["Machine","Week"])
print("\nMachine-week summary:")
print(df_mach.to_string(index=False))


Status: Optimal

Total overtime over both weeks = 142.4351 hours

Part-level summary (with inventory):
 Part  Demand_week1  Demand_week2  Prod_w1_good  Prod_w2_good  Inventory_carry
    1          3500          3000   3500.000028   2999.999982              0.0
    2          3000          2800   3000.000000   2799.999999              0.0
    3          4000          4000   4000.000125   4000.000125              0.0
    4          4000          4300   3999.999900   4299.999990              0.0
    5          2800          2800   3000.000000   2600.000010            200.0

Machine-week summary:
 Machine  Week  Prod_hours  Setup_hours_effective  Total_hours        OT
       1     1  151.999997                   16.0   167.999997 48.000000
       1     2  120.000000                    0.0   120.000000  0.000000
       2     1  110.168500                   10.0   120.168500  0.168498
       2     2  111.999999                    8.0   119.999999  0.000000
       3     1  133.595960         

In [10]:
# OR 6205 – Falcon Die Casting
# Question 9: Two-week model (Weeks 11 & 12) with overtime, inventory, and backlog

!pip install pulp
import pulp
import pandas as pd

# -----------------------------
# 1. Data
# -----------------------------
machines = [1,2,3,4,5]
parts = [1,2,3,4,5]
weeks = [1,2]  # 1 = Week 11, 2 = Week 12

# Demand for Week 11 and Week 12
demand = {
    (1,1): 4500, (2,1): 4000, (3,1): 5000, (4,1): 5000, (5,1): 3800,  # week 11
    (1,2): 3000, (2,2): 2800, (3,2): 4000, (4,2): 4300, (5,2): 2800   # week 12
}

# Production rates (units/hour) – same as before
rates = {
    (1,1):40, (1,4):60,
    (2,1):35, (2,2):25,
    (3,2):30, (3,5):45,
    (4,2):35, (4,3):50,
    (5,4):60, (5,5):50
}

# Yields
yields = {1:0.60, 2:0.55, 3:0.75, 4:0.65, 5:0.60}

# Setup times (hours)
setups = {
    (1,1):8, (1,4):8,
    (2,1):10, (2,2):8,
    (3,2):10, (3,5):24,
    (4,2):8, (4,3):12,
    (5,4):8, (5,5):20
}

REG = 120
MAXOT = 48
BIGM = 10000

# Costs
c_ot = 30.0   # $/overtime hour
c_inv = 2.0   # $/unit inventory per week
c_bk  = 3.0   # $/unit backlog per week

# Effective production rates (good units/hour)
eff = {(m,p): rates[(m,p)] * yields[p] for (m,p) in rates.keys()}

# -----------------------------
# 2. Model
# -----------------------------
model = pulp.LpProblem("FDC_Q9_TwoWeek_Backlog_Inventory", pulp.LpMinimize)

# Decision variables
h = pulp.LpVariable.dicts("h", (machines, parts, weeks), lowBound=0)
x = pulp.LpVariable.dicts("x", (machines, parts, weeks), lowBound=0, upBound=1, cat='Binary')
OT = pulp.LpVariable.dicts("OT", (machines, weeks), lowBound=0, upBound=MAXOT)

I = pulp.LpVariable.dicts("Inv", (parts, weeks), lowBound=0)     # inventory end of week t
B = pulp.LpVariable.dicts("Backlog", (parts, weeks), lowBound=0) # backlog end of week t

# -----------------------------
# 3. Objective
# -----------------------------
model += (
    c_ot  * pulp.lpSum(OT[m][t] for m in machines for t in weeks)
  + c_inv * pulp.lpSum(I[p][t] for p in parts for t in weeks)
  + c_bk  * pulp.lpSum(B[p][t] for p in parts for t in weeks)
), "Total_Cost"

# -----------------------------
# 4. Constraints
# -----------------------------

# Inventory-backlog flow balance for each part and week
for p in parts:
    # Initial conditions I_{p,0} = B_{p,0} = 0
    prev_I = 0.0
    prev_B = 0.0
    for t in weeks:
        model += (
            I[p][t] - B[p][t]
            == prev_I - prev_B
            + pulp.lpSum(eff[(m,p)] * h[m][p][t]
                         for m in machines if (m,p) in eff)
            - demand[(p,t)]
        ), f"flow_p{p}_w{t}"

        # Update prev_I, prev_B for next week
        prev_I = I[p][t]
        prev_B = B[p][t]

# Production only if setup is used in that week
for (m,p) in eff:
    for t in weeks:
        model += h[m][p][t] <= BIGM * x[m][p][t], f"prod_link_m{m}_p{p}_w{t}"

# Machine capacity each week (no setup carry-over)
for m in machines:
    for t in weeks:
        model += (
            pulp.lpSum(h[m][p][t] for p in parts if (m,p) in eff) +
            pulp.lpSum(setups[(m,p)] * x[m][p][t]
                       for p in parts if (m,p) in setups)
            == REG + OT[m][t]
        ), f"time_balance_m{m}_w{t}"

# -----------------------------
# 5. Solve
# -----------------------------
model.solve(pulp.PULP_CBC_CMD(msg=1))
print("Status:", pulp.LpStatus[model.status])

total_OT = sum(OT[m][t].value() for m in machines for t in weeks)
total_I  = sum(I[p][t].value() for p in parts for t in weeks)
total_B  = sum(B[p][t].value() for p in parts for t in weeks)
total_cost = (
    c_ot  * total_OT +
    c_inv * total_I +
    c_bk  * total_B
)

print(f"\nTotal overtime = {total_OT:.4f} hours")
print(f"Total inventory across weeks = {total_I:.4f} units")
print(f"Total backlog across weeks = {total_B:.4f} units")
print(f"Total cost = ${total_cost:.2f}")

# -----------------------------
# 6. Part-week summary
# -----------------------------
rows_parts = []
for p in parts:
    for t in weeks:
        produced = sum(
            eff[(m,p)] * h[m][p][t].value()
            for m in machines if (m,p) in eff
        )
        rows_parts.append({
            "Part": p,
            "Week": t,
            "Demand": demand[(p,t)],
            "Produced_good": produced,
            "Inv_end": I[p][t].value(),
            "Backlog_end": B[p][t].value()
        })

df_parts = pd.DataFrame(rows_parts).sort_values(["Part","Week"])
print("\nPart-week summary:")
print(df_parts.to_string(index=False))

# -----------------------------
# 7. Machine-week summary
# -----------------------------
rows_mach = []
for m in machines:
    for t in weeks:
        prod_h = sum(h[m][p][t].value() for p in parts if (m,p) in eff)
        setup_h = sum(setups[(m,p)] * x[m][p][t].value()
                      for p in parts if (m,p) in setups)
        total_h = prod_h + setup_h
        rows_mach.append({
            "Machine": m,
            "Week": t,
            "Prod_hours": prod_h,
            "Setup_hours": setup_h,
            "Total_hours": total_h,
            "OT": OT[m][t].value()
        })

df_mach = pd.DataFrame(rows_mach).sort_values(["Machine","Week"])
print("\nMachine-week summary:")
print(df_mach.to_string(index=False))


Status: Optimal

Total overtime = 423.7888 hours
Total inventory across weeks = 0.0000 units
Total backlog across weeks = 1401.5898 units
Total cost = $16918.43

Part-week summary:
 Part  Week  Demand  Produced_good  Inv_end  Backlog_end
    1     1    4500    4209.076920      0.0    290.92308
    1     2    3000    3290.923077      0.0      0.00000
    2     1    4000    2889.333340      0.0   1110.66670
    2     2    2800    3910.666667      0.0      0.00000
    3     1    5000    4999.999875      0.0      0.00000
    3     2    4000    4000.000125      0.0      0.00000
    4     1    5000    4999.999797      0.0      0.00000
    4     2    4300    4299.999990      0.0      0.00000
    5     1    3800    3800.000100      0.0      0.00000
    5     2    2800    2799.999990      0.0      0.00000

Machine-week summary:
 Machine  Week  Prod_hours  Setup_hours  Total_hours        OT
       1     1  151.999995         16.0   167.999995 48.000000
       1     2  152.000000         16.0   1